In [1]:
import numpy as np 
import pandas as pd
import wrds
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from scipy import stats
from scipy.stats import rankdata
import scipy.stats as st
import statsmodels.api as sm
import plotly.express as px

In [ ]:
db = wrds.Connection(wrds_username='malachinelson')

In [ ]:
# Pull Raw Data From Compustat (Note: Only need to do once)
compustat_qraw = db.raw_sql('''
                            select gvkey, tic, exchg, fyearq, fdateq, actq, aoq, atq, lctq, loq, lseq, ltq, niq, revtq
                                , prccq, cshoq
                            from comp.fundq
                            where fyearq >= 2021
                            ''')
compustat_qraw = compustat_qraw.sort_values(by = ['gvkey','fyearq']).dropna().drop_duplicates()
# compustat_qraw.to_csv('C:/Users/nelso/Downloads/Data/compustat_qraw.csv')

# Getting 2022 Data by Using Last 4 Quarters Available

In [8]:
compustat_qraw = pd.read_csv("C:/Users/nelso/Downloads/Data/compustat_qraw.csv")

In [9]:
## Pull Last 4 Quarters and Most Recent Quarter from Compustat Data
compustat_last4q = compustat_qraw.groupby(['tic']).tail(4).reset_index()
compustat_last1q = compustat_qraw.groupby(['tic']).tail(1).reset_index()
## Pull Our 2021 Ticker List (Will be used as proxy for 2022 Ticker List)
full_lagged_ni_growth = pd.read_csv("C:/Users/nelso/Downloads/Data/full_lagged_ni_growth.csv")
ticker_list = full_lagged_ni_growth[full_lagged_ni_growth['year']==2021]['ticker']
ticker_list = pd.DataFrame(data=ticker_list).rename({'ticker': "tic"}, axis=1)
## Sum Last 4 Quarters Data to Get Annualized Data
compustat_last4q_nisum = compustat_last4q.groupby('tic')['niq'].agg(['sum','count']).reset_index()
## Inner Join Compustat Data to Our Ticker List and Make Sure Each Ticker has 4 Quarters of Data
new_list = ticker_list.merge(compustat_last4q_nisum ,on='tic', how='left')
new_list = new_list[new_list['count']==4].rename({'sum': "ni"}, axis=1)
## Merge with Most Recent Quarter Data to Get Most Recent Asset Total, Price, and Shares Outstanding
final_list = pd.merge(new_list, compustat_last1q[['tic','atq','revtq','prccq','cshoq']], on='tic', how='inner').rename({'atq': "at",'revtq': 'revt'}, axis=1)
final_list['fyear'] = 2022

# Data Set-Up (Pulled from FF_FM Code)

In [10]:
## Rerunning CAGRs with New 2022 Data 
fin = pd.read_csv("C:/Users/nelso/Downloads/Data/compustat_data.csv")
fin['fyear'] = pd.to_datetime(fin.fyear, format='%Y').dt.year
fin = fin.drop(['Unnamed: 0'],axis=1)
## Removing Any Existing 2022 Data from Dataframe
fin = fin[fin['fyear']!=2022]
fin = pd.concat([fin,final_list])
fin = fin.sort_values(['tic','fyear'], ascending=[True,True])
fin['fyear+10'] = fin['fyear'] + 10
fin_new = pd.merge(fin, fin, how='inner', left_on=['tic', 'fyear'], right_on=['tic','fyear+10'], suffixes=['_end', '_start'])
fin_new = fin_new.drop(['fyear+10_start','fyear+10_end'],axis=1)
fin_new = fin_new[fin_new['ni_start'] != 0]
fin_new = fin_new[fin_new['ni_end'] != 0]
fin_new['CAGR10_ni'] = (fin_new['ni_end']/fin_new['ni_start'])**(0.1)-1
fin_new['CAGR10_assets'] = (fin_new['at_end']/fin_new['at_start'])**(0.1)-1
fin_new['CAGR10_rev'] = (fin_new['revt_end']/fin_new['revt_start'])**(0.1)-1
fin_new['ni/at'] = fin_new['ni_start']/fin_new['at_start']
fin_CAGR = fin_new[['tic','fyear_end','CAGR10_ni','CAGR10_assets','CAGR10_rev','ni/at','prccq_end','cshoq_end']]

##Pulling Out Only 2022 Data
fin_CAGR22 = fin_CAGR[fin_CAGR['fyear_end']==2022]
ticker_list2 = full_lagged_ni_growth[full_lagged_ni_growth['year']==2021][['ticker','permno']]
ticker_list2 = pd.DataFrame(data=ticker_list2).rename({'ticker': "tic"}, axis=1)
fin_CAGR22_new = ticker_list2.merge(fin_CAGR22 ,on='tic', how='left').rename({'tic': "ticker"}, axis=1)
fin_CAGR22_new['shrout'] = fin_CAGR22_new['cshoq_end']*1000000
fin_CAGR22_new['mcap'] = (fin_CAGR22_new['shrout'] * fin_CAGR22_new['prccq_end'])/1000000000
fin_CAGR22_new['mcap_inf'] = fin_CAGR22_new['mcap']
fin_CAGR22_new['inflation_multiple'] = 0
fin_CAGR22_new['year'] = fin_CAGR22_new['fyear_end']

In [11]:
## Pulling Returns, Cleaning Data, Ranking CAGRs
ret = pd.read_csv('C:/Users/nelso/Downloads/Data/crsp_data_clean.csv')
inflation_factors = pd.read_csv('C:/Users/nelso/Downloads/Data/inflation_multiples.csv')
ret = ret[ret['year']!=2022]
final_data_set = pd.merge(ret,fin_CAGR, left_on=['year','ticker'], right_on=['fyear_end','tic']).rename({'return.1': "annual_return"}, axis=1)
sec_info = pd.read_csv('C:/Users/nelso/Downloads/Data/sec_info.csv.gz')
sec_info = sec_info[(sec_info['SHRCD'] == 10)  | (sec_info['SHRCD'] == 11) | (sec_info['SHRCD'] == 12) ]
sec_info = sec_info[(sec_info['EXCHCD'] == 2)  | (sec_info['EXCHCD'] == 1) | (sec_info['EXCHCD'] == 3) ]
sec_info = sec_info[['TICKER']].drop_duplicates()
final_dataset2 = pd.merge(final_data_set, sec_info['TICKER'] , left_on=['ticker'], right_on=['TICKER'], how='inner')
final_dataset_merge = final_dataset2.merge(inflation_factors, on='year')
final_dataset_merge['mcap_inf']= final_dataset_merge['mcap']* final_dataset_merge['inflation_multiple']
final_dataset_merge = pd.concat([final_dataset_merge,fin_CAGR22_new])
final_dataset_merge = final_dataset_merge[(final_dataset_merge['mcap_inf'] > .5)]
final_dataset_merge = final_dataset_merge[(final_dataset_merge['ni/at'] > .0015)]
final_dataset_merge['CAGR10_ni_rank'] = final_dataset_merge.groupby(['fyear_end'])['CAGR10_ni'].rank(ascending=False)
final_dataset_merge = final_dataset_merge[['year','ticker','permno','annual_return','mcap','inflation_multiple','mcap_inf','mcap_rank','CAGR10_ni','CAGR10_ni_rank','CAGR10_assets','CAGR10_rev']]

stock_list2022 = final_dataset_merge[final_dataset_merge['year']==2022][['year','ticker','permno','mcap_inf','CAGR10_ni','CAGR10_ni_rank']]

# Stock Selection

In [12]:
def top_rank_companies(df, n, time, identification, rank):    
    df_copy = df.copy().set_index([identification])
    df = df.copy().set_index([time, identification]).\
            loc[df_copy.groupby(time)[rank].nsmallest(n).index]
    return df.reset_index()

In [15]:
top30_2022 = top_rank_companies(stock_list2022, 30, 'year', 'ticker', 'CAGR10_ni_rank')
top30_2022

,year,ticker,permno,mcap_inf,CAGR10_ni,CAGR10_ni_rank
0,2022.0,META,21413,361.587200,0.877413,1.0
1,2022.0,CPE,80926,2.156861,0.840120,2.0
2,2022.0,NFLX,89393,104.775509,0.765414,3.0
3,2022.0,QDEL,76591,4.717680,0.663402,4.0
4,2022.0,RRC,50017,6.120094,0.579986,5.0
5,2022.0,MOH,89781,19.130720,0.560623,6.0
6,2022.0,VRTV,14751,1.319895,0.536745,7.0
7,2022.0,AMAT,14702,74.516760,0.505620,8.0
8,2022.0,CVCO,89792,1.833116,0.489491,9.0
9,2022.0,CRTO,14240,1.620070,0.484633,10.0


In [7]:
top30_2022 = top_rank_companies(stock_list2022, 30, 'year', 'ticker', 'CAGR10_ni_rank')
top30_2022
top30_2022.to_csv('C:/Users/nelso/Downloads/Data/top30_2022.csv')